In [1]:
# Importing the libraries
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import confusion_matrix
import statistics
from sklearn.impute import SimpleImputer

In [2]:
def cmatrix(cm):
    val=0
    for i in range (3):
        val+=cm[i][i]
    print(val/sum(sum(cm))*100)

In [3]:
data_train=pd.DataFrame(pd.read_csv('/content/train_yaOffsB.csv'))

In [4]:
df=data_train.copy()
df.drop('ID',axis=1,inplace=True)

In [5]:
df.head()

,Estimated_Insects_Count,Crop_Type,Soil_Type,Pesticide_Use_Category,Number_Doses_Week,Number_Weeks_Used,Number_Weeks_Quit,Season,Crop_Damage
0,188,1,0,1,0,0.0,0,1,0
1,209,1,0,1,0,0.0,0,2,1
2,257,1,0,1,0,0.0,0,2,1
3,257,1,1,1,0,0.0,0,2,1
4,342,1,0,1,0,0.0,0,2,1


In [24]:
X = df.iloc[:, 0:8].values
y = df.iloc[:, 8].values

In [25]:
X.shape

(88858, 8)

In [33]:
imputer = SimpleImputer(missing_values=np.nan, strategy='constant')
imputer = imputer.fit(X)
X = imputer.transform(X)

In [34]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.1, random_state = 0)

In [35]:
classifier1 = RandomForestClassifier(n_estimators =500, criterion = 'entropy', random_state = 0)
classifier1.fit(X_train, y_train)
classifier2 = RandomForestClassifier(n_estimators = 500, criterion = 'entropy', random_state = 0)
classifier2.fit(X_train, y_train)
classifier3 = RandomForestClassifier(n_estimators = 500, criterion = 'gini', random_state = 0)
classifier3.fit(X_train, y_train)

RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=None, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=500,
                       n_jobs=None, oob_score=False, random_state=0, verbose=0,
                       warm_start=False)

In [36]:
y_pred1 = classifier1.predict(X_test)
y_pred2 = classifier2.predict(X_test)
y_pred3 = classifier3.predict(X_test)
y_pred=np.around((y_pred1+y_pred2+y_pred3)/3).astype(int)

In [37]:
cm = confusion_matrix(y_test, y_pred)
cmatrix(cm)

82.06167004276391


In [38]:
def pred_class(X):
    y_pred1 = classifier1.predict(X)
    y_pred2 = classifier2.predict(X)
    y_pred3 = classifier3.predict(X)
    y_pred=np.around((y_pred1+y_pred2+y_pred3)/3).astype(int)
    return y_pred

In [39]:
data_test=pd.DataFrame(pd.read_csv('/content/test_pFkWwen.csv'))
df_test=data_test.copy()
col1=pd.DataFrame(df_test['ID'])
X_test = df_test.iloc[:, 1:9].values
imputer = imputer.fit(X_test)
X_test = imputer.transform(X_test)

In [ ]:
y_pred=pred_class(X_test)
col2=pd.DataFrame(y_pred)
col2.rename(columns={0: "Crop_Damage"},inplace=True)

In [ ]:
result=pd.DataFrame({"ID":col1['ID'],"Crop_Damage":col2['Crop_Damage']}).set_index(['ID']).to_csv('submission.csv')